---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
## import matplotlib.pyplot as plt

from networkx.algorithms import bipartite

## %matplotlib notebook


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);
        
Data = pd.read_csv('Employee_Movie_Choices.txt', sep='\t', header=None, skiprows = 1, names=['Employees', 'Movies'])
Rela = pd.read_csv('Employee_Relationships.txt', sep='\t', header=None, skiprows = 1, names=['1', '2', 'Score'])

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
## plt.figure(figsize=(10,15))
## edges = G.edges()
## print(edges)
## X, Y = bipartite.sets(G)
## pos = dict()
## pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
## pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
## nx.draw_networkx(G, pos=pos, edges=edges)
## plt.show()

In [3]:
def answer_one():
        
    G = nx.Graph()

    G.add_nodes_from(Data['Employees'], bipartite=0)
    G.add_nodes_from(Data['Movies'], bipartite=1)

    G = nx.from_pandas_edgelist(Data, 'Employees', 'Movies', edge_attr=None)
    
    return G

In [4]:
answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [5]:
def answer_two():
    
    G = answer_one()
    G.add_nodes_from(Data['Employees'], bipartite=0, type = 'employee') 
    G.add_nodes_from(Data['Movies'], bipartite=1, type = 'movie')
    
    return G

In [6]:
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [7]:
def answer_three():
        
    G_employees = bipartite.weighted_projected_graph(answer_two(),employees)
    
    return G_employees

In [8]:
answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [9]:
def answer_four():
        
    Weight = nx.to_pandas_edgelist(answer_three())
    Weight.rename(columns={'source':'1','target':'2','weight':'Weight'}, inplace=True)
    Weight_t = pd.DataFrame({'1':Weight['2'],'2':Weight['1'],'Weight':Weight['Weight']})
    weight = pd.concat([Weight, Weight_t])

    Merge = pd.merge(Rela, weight,  how='left', left_on=['1','2'], right_on = ['1','2'])
    Merge['Weight'] = Merge['Weight'].fillna(0)
    Corre = Merge['Score'].corr(Merge['Weight'])
    
    return Corre

In [10]:
answer_four()

0.8014749455697414

In [11]:
## answer_three().edges(data=True)
## nx.to_pandas_adjacency(answer_three(), answer_three().nodes(), dtype=int)